In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io.arff import loadarff
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import MinMaxScaler

# Load the data
data = loadarff('data/pd_speech.arff')
df = pd.DataFrame(data[0])
df['class'] = df['class'].str.decode('utf-8')

In [18]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Normalize the data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Apply k-means clustering
kmeans = [KMeans(n_clusters=3, random_state=seed) for seed in range(3)]
predictions = [kmeans[seed].fit_predict(X) for seed in range(3)]

In [19]:
# assess k-means silhouette scores
silhouette_scores = [silhouette_score(X, predictions[seed]) for seed in range(3)]
print('Silhouette scores: {}'.format(silhouette_scores))

Silhouette scores: [0.6061811635543617, 0.6061811635543617, 0.6061811635543617]
